In [2]:
!pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/14.9 MB 9.1 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/14.9 MB 8.0 MB/s eta 0:00:02
   -------------- ------------------------- 5.2/14.9 MB 8.6 MB/s eta 0:00:02
   ------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [3]:
import spacy

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from spacy.util import minibatch
from spacy.training import Example
import random

In [9]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [10]:
df = df[['text', 'airline_sentiment']]
df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [11]:
df['airline_sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [12]:
df = df[df['airline_sentiment'] != 'neutral']
df['airline_sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [13]:
df.isna().sum()

text                 0
airline_sentiment    0
dtype: int64

In [14]:
df['airline_sentiment'] = df['airline_sentiment'].map({'negative': 0, 'positive': 1})

In [15]:
df['airline_sentiment'].unique()

array([1, 0])

In [17]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 8.4 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 9.1 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 8.6 MB/s eta 0:00:01
     ---------------------- ----------------- 7.3/12.8 MB 9.1 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.8 MB 9.2 MB/s eta 0:00:01
     ----------------------------------- ---- 11.3/12.8 MB 9.4 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 9.2 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
text = "This is an example text which describe the classification of the text."
nlp = spacy.load("en_core_web_sm")

In [20]:
doc = nlp(text)

In [21]:
for token in doc:
    print(token.text)

This
is
an
example
text
which
describe
the
classification
of
the
text
.


In [22]:
ex = "organizie organize and organizing"
for token in nlp(ex):
    print(token.text, token.lemma_)

organizie organizie
organize organize
and and
organizing organize


In [33]:
#Load spacy model
nlp = spacy.blank("en")


In [34]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['airline_sentiment'], test_size=0.2, random_state=42)

In [35]:
Config = {
    "threshold": 0.5,
    "model": {
        "@architectures": "spacy.TextCatEnsemble.v2",
        "linear_model": {
            "@architectures": "spacy.TextCatBOW.v2",
            "exclusive_classes": True,
            "ngram_size": 1,
            "no_output_layer": False
        },
        "tok2vec": {
            "@architectures": "spacy.Tok2Vec.v2",
            "embed": {
                "@architectures": "spacy.MultiHashEmbed.v2",
                "width": 96,
                "rows": [2000, 2000, 100, 100, 100],
                "attrs": ["NORM", "LOWER", "PREFIX", "SUFFIX", "SHAPE"]
            },
            "encode": {
                "@architectures": "spacy.MaxoutWindowEncoder.v2",
                "width": 96,
                "window_size": 4,
                "maxout_pieces": 3,
                "depth": 4
            }
        }
    }
}

In [36]:
textcat = nlp.add_pipe("textcat", config=Config)
textcat.add_label("positive")
textcat.add_label("negative")  

1

In [37]:
#Define the training data
train_data = list(zip(X_train, [{"cats": {"positive": bool(y), "negative": not bool(y)}} for y in y_train]))
random.shuffle(train_data)

In [38]:
from sklearn.metrics import classification_report  

In [39]:
optimizer = nlp.begin_training()

for epoch in range(1):
    random.shuffle(train_data)
    losses = {}
    # Create the minibatches
    for batch in minibatch(train_data, size=8):
        texts, annotations = zip(*batch)
        examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in zip(texts, annotations)]
        nlp.update(examples, drop=0.5, losses=losses)
    print(f"Epoch {epoch + 1}, Loss: {losses['textcat']}")

y_pred = [nlp(text).cats[positive] > 0.5 for text in X_test]  
print(classification_report(y_test, y_pred, target_names=['negative', 'positive']))


Epoch 1, Loss: 146.30238383470714


NameError: name 'positive' is not defined